In [ ]:
from ase.io import read
atoms = read('/home/phanim/harshitrawat/summer/universal_embeddings_results/Universal_on_T3.xyz', index=0)   # or loop indices
latents = atoms.arrays['mace_latent']          # shape (n_atoms, 256)
# structure-level embedding:
struct_emb = latents.mean(axis=0)

In [10]:
from ase.io import read
atoms = read('/home/phanim/harshitrawat/summer/universal_embeddings_results/Universal_on_T3.xyz', index=0)   # or loop indices
keys = atoms.arrays.keys()
print(keys)  # print available array keys

dict_keys(['numbers', 'positions', 'spacegroup_kinds', 'REF_forces', 'mace_latent'])


In [3]:
print(struct_emb.shape)  # should be (256,)
print(latents.shape)     # should be (n_atoms, 256)

(256,)
(648, 256)


In [ ]:
print(latents[0])  # print latent vector of first atom

In [5]:
energy = atoms.info['mace_energy']  # get energy from info dictionary
print(energy)  # print the energy

-2678.5194428487684


For all the .xyz files in /home/phanim/harshitrawat/summer/universal_embeddings_results we will save a .json 
which will be utilised for clustering the latents
which will have 1. file name 2. structure-level latent embedding by mean pooling 3. mace_energy

In [7]:
import os
for files in os.listdir('/home/phanim/harshitrawat/summer/universal_embeddings_results/'):
    if files.endswith('.xyz'):
        print(files)

Universal_on_T1.xyz
Universal_on_T2.xyz
Universal_on_T3.xyz


In [ ]:
import os
import json
from ase.io import read

BASE_DIR = "/home/phanim/harshitrawat/summer/universal_embeddings_results"
OUT_JSON = os.path.join(BASE_DIR, "structure_level_latents.json")

results = []

for fname in sorted(os.listdir(BASE_DIR)):
    if not fname.endswith(".xyz"):
        continue

    xyz_path = os.path.join(BASE_DIR, fname)

    # read ALL structures in the xyz file
    atoms_list = read(xyz_path, index=":")

    for atoms in atoms_list:
        struct_file = atoms.info["file"]          # original structure filename
        energy = atoms.info["mace_energy"]         # structure energy

        latents = atoms.arrays["mace_latent"]      # (N_atoms, D)
        struct_emb = latents.mean(axis=0)          # (D,)

        results.append(
            {
                "file": struct_file,
                "structure_embedding": struct_emb.tolist(),
                "mace_energy": energy,
            }
        )

with open(OUT_JSON, "w") as f:
    json.dump(results, f, indent=2)

print(f"Saved {len(results)} structures to {OUT_JSON}")
